# Covid Data Preparation

In diesem Notebook werden die Daten aufbereitet.

## Imports and Files

In [ ]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline

PATH_COLLECTED_DATA = './collected_data.csv'

In [ ]:
# define function to help remove missing values
def addNumberOfMissingValuesToDataframe(dataframe):
    missing_values = []
    for i in range(len(dataframe.index)):
        missing_values.append(dataframe.iloc[i].isnull().sum())
    dataframe['missing_values'] = missing_values
    return dataframe

In [ ]:
# read data
df = pd.read_csv(PATH_COLLECTED_DATA)

df.drop(['Female share of employment in senior and middle management (%)','Firms with female participation in ownership (% of firms)', 'Women who were first married by age 18 (% of women ages 20-24)', 'International tourism, number of departures', 'overweight_in_percent', 'stunting_in_percent', 'underweight_in_percent'],inplace=True, axis=1)

print(df.shape)
fig, ax = plt.subplots(figsize=(20,5))
sns.heatmap(data = df.isnull(), yticklabels=False, cbar=False, cmap='viridis', ax=ax)

In [ ]:
# correlation and removal
corr = df.loc[:, ['di_score', 'di_electoral_process_and_pluralism', 'di_functioning_of_government', 'di_political_participation', 'di_political_culture', 'di_civil_liberties', 'household_avg_people', 'household_members_1', 'household_members_2_3', 'household_members_4_5', 'household_members_6_or_more']].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

plt.figure(figsize=(12,10))
sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
# drop di values because of correlation
# drop household values because of correlation
df.drop(['di_electoral_process_and_pluralism', 'di_functioning_of_government', 'di_political_participation', 'di_political_culture', 'di_civil_liberties', 'di_regime_type', 'household_members_1', 'household_members_2_3', 'household_members_4_5', 'household_members_6_or_more'],inplace=True, axis=1)

corr = df.corr()

corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# delete columns with strong correlation
df.drop(['GNI per capita, Atlas method (current US$)', 'GNI per capita, PPP (current international $)', 'GNI, PPP (current international $)', 'GDP growth (annual %)', 'handwashing_facilities', 'International tourism, expenditures (current US$)', 'International tourism, number of arrivals', 'International tourism, receipts (current US$)'], inplace=True, axis=1)

corr = df.corr()

corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# combine female and male values
FEMALE_RATIO = 0.495
MALE_RATIO = 1 - FEMALE_RATIO

# 'female_smokers', 'male_smokers', 'Life expectancy at birth, female (years)', 'Life expectancy at birth, male (years)', 'Wage and salaried workers, female (% of female employment) (modeled ILO estimate)', 'Wage and salaried workers, male (% of male employment) (modeled ILO estimate)', 'Mortality rate, adult, female (per 1,000 female adults)', 'Mortality rate, adult, male (per 1,000 male adults)'
df['smokers_percentage'] = FEMALE_RATIO * df['female_smokers'] + MALE_RATIO * df['male_smokers']
df['life_expectancy_at_birth'] = FEMALE_RATIO * df['Life expectancy at birth, female (years)'] + MALE_RATIO * df['Life expectancy at birth, male (years)']
df['wage_and_salaried_workers_percentage_employed'] = FEMALE_RATIO * df['Wage and salaried workers, female (% of female employment) (modeled ILO estimate)'] + MALE_RATIO * df['Wage and salaried workers, male (% of male employment) (modeled ILO estimate)']
df['mortality_rate_per_1000'] = FEMALE_RATIO * df['Mortality rate, adult, female (per 1,000 female adults)'] + MALE_RATIO * df['Mortality rate, adult, male (per 1,000 male adults)']


# mortality rate and life expectancy strongly correlate -> drop mortality_rate_per_1000
# same with wage_and_salaried_workers_percentage_employed -> drop life_expectancy_at_birth because workers have a high probabilty of contact with others

df.drop(['female_smokers', 'male_smokers', 'Life expectancy at birth, female (years)', 'Life expectancy at birth, male (years)', 'Wage and salaried workers, female (% of female employment) (modeled ILO estimate)', 'Wage and salaried workers, male (% of male employment) (modeled ILO estimate)', 'Mortality rate, adult, female (per 1,000 female adults)', 'Mortality rate, adult, male (per 1,000 male adults)', 'mortality_rate_per_1000', 'life_expectancy_at_birth'], inplace=True, axis=1)

corr = df.corr()

corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# replenish rows missing only some values

In [ ]:
# delete colums missing to many values
addNumberOfMissingValuesToDataframe(df)
treshold = 10
df = df[df.missing_values < treshold]